In [1]:
#importation des packagins : 

import pandas as pd
import numpy as np 


#importation des packaging de visualisation : 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#importation des outils statistiques :
from scipy.stats import chi2_contingency as khi2

#importation des packaging de preprocessing : 
from sklearn.preprocessing import OrdinalEncoder

#importation des packaging de pour la selection du meilleur model :
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

#importation des models de Classification : 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

#importation des packagins pour de la feature selection : 
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline


#importation des metrics : 
from sklearn.metrics import accuracy_score, make_scorer

import warnings
warnings.filterwarnings('ignore')

Plan : 
 - 1. Création du modèle 
 - 2. Mesure de la performence du modèle 
 - 3. Amélioration du modèle
    - select features, gridsearch, ..
 - 4. mesure performence du modèle

In [2]:
data = pd.read_csv("dataset_nettoyes.csv")

In [3]:
X, y = data.drop('hospital_death',axis = 1), data['hospital_death']


In [4]:
X.head(1)

,elective_surgery,ethnicity,icu_admit_source,icu_stay_type,icu_type,apache_post_operative,arf_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,...,h1_mbp_noninvasive_min,h1_resprate_max,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,d1_glucose_max,d1_potassium_max,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,0,2.0,1.0,0.0,2.0,0,0.0,3.0,6.0,0.0,...,0.293586,0.469044,-0.079922,-0.050362,-0.06815,-0.05582,-0.060371,-0.389735,0.069252,0.039076


In [5]:
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: hospital_death, dtype: int64

#  1. Création du modèle 

In [28]:
#instanciation du modèle sans regler les hypers parametres
knn = KNeighborsClassifier()

In [7]:
#preparation des splits a faire
n_cross = KFold(n_splits=5, shuffle=True, random_state=42)

In [29]:
#resultat de l'accuracy pour chaque split : 
accuracy = cross_val_score(estimator=knn, X=X, y=y, scoring = make_scorer(accuracy_score),
                cv = n_cross)

#  2. Mesure de la performence du modèle 


In [30]:
accuracy_moyenne = accuracy.mean()
accuracy_moyenne

0.9185284503362914

#  3. Amélioration du modèle
- 1. Réduction du nombre de colonne **manuellement**
- 2. Utilisation du GridSearchCV : 

## 3.1
On remarquera que le code est **long a s'executer**. Par consequent, nous allons **réduire le nombre de colonne pour faciliter l'execution des algorithmes**.

- Dans notre notebook 001_Patient_preprocessing, nous avions garder 59 colonnes grâce à l'analyse bi-variée.
- Pour des raisons pratiques, on va supprimer un nombre conséquent de colonne afin d'accélerer la création de l'algorithme.
- Dans la partie analyse-bivariée de notre notebook de preprocessing, on remarque que pour :
    - l'analyse des boxplot, qu'**aucuns boxplot n'est vraiment pertinent** donc nous n'allons garder aucunes variables numériques.
    - test chi2, on a récupérer 22 variables, on va prendre uniquement **les 8 variables les plus corrélés**.

In [43]:
feats_reduit = ['gcs_eyes_apache','gcs_motor_apache','gcs_verbal_apache','intubated_apache','ventilated_apache',
 "apache_3j_bodysystem","icu_admit_source","apache_2_bodysystem"]

In [45]:
X = X[feats_reduit]

In [49]:
accuracy = cross_val_score(estimator=knn, X=X, y=y, scoring = make_scorer(accuracy_score),
                cv = n_cross)

## 3.2

In [54]:
parametre_grid =  {"n_neighbors":np.arange(2,20)}

In [55]:
grid = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=parametre_grid, 
             scoring= make_scorer(accuracy_score), cv = n_cross)

# Attention : le code ci-dessous est  long a s'executer 

In [56]:
grid.fit(X,y)

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])},
             scoring=make_scorer(accuracy_score))

In [59]:
model = grid.best_estimator_

In [60]:
accuracy_grid = cross_val_score(estimator=model, X=X, y=y, scoring = make_scorer(accuracy_score),
                cv = n_cross)

In [61]:
accuracy_grid_moy = accuracy_grid.mean()

In [62]:
accuracy_grid_moy
#avec 8 colonnes on a un moins bon score malgré qu'on ai combiné au GridSearch

0.9148975673474407

# - 4. mesure performence du modèle